In [ ]:
import gc
import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as constants

from matplotlib.ticker import FormatStrFormatter
from tqdm.notebook import tqdm
from datetime import datetime
from astropy import units as u

import Hirogen_Functions

np.seterr(invalid='raise')


startT = time.time()

#######################
# NERSC Flag
#######################

# Flags the script as running on NERSC - i.e. on DESI spectra
# This activates changes in the script to handle the DESI spectra as well as disabling any multithreading for the
# time being.

NERSC_Flag = False
Local_DESI = False
# Both flags should be False to run on local SDSS spectra

if not NERSC_Flag:

    User = 'Joe'
    User_Config = Hirogen_Functions.user_config(user=User)

    Database = User_Config[0]
    Database_User = User_Config[1]
    Database_Password = User_Config[2]
    Main_Spectra_Path = User_Config[3]

    Standard_Path_Flag = True # This should be set to False if the spectra are in a non standard location - Only applies to SDSS spectra

    # Restrictions on the which objects are retrieved are set at the actual request point
    # Search for 'QUERY' to be taken to it

    #########
    # Tables
    ########

    # TableID = 'SDSS_Confirmed_Objects'

    # TableID = "SDSS_Test_QSO_Sample"
    # TableID = "SDSS_Test_Sample"

    TableID = 'SDSS_Followup_Spectra'
    # TableID = "SDSS_CL_QSOs

    # TableID = 'Dark4_GTest5'
    # BLOCK_ID = 'SV3_Dark_4_Big_GSmooth_5' #DESI Only

    #TableID = 'DESI_LOCAL'
    #TableID = 'DESI_LOCAL_Interesting_Bright_Block_3'

    # TableID = "SDSS_Confirmed_Objects_Rebin_Test"
    # TableID = "SDSS_Test_Sample_GSmooth"
    # TableID = "SDSS_Confirmed_Objects_SavGol"

else:
    
    import desispec.coaddition as co_add

    #from desitarget.cmx.cmx_targetmask import cmx_mask
    #from desitarget.targetmask import desi_mask
    #from desitarget.sv1.sv1_targetmask import bgs_mask # < This is the current mask for BGS Targets

    from desispec.io import read_spectra, write_spectra
    from desispec.coaddition import coadd_cameras
    from dust_extinction.parameter_averages import F99
    from collections import Counter, defaultdict
    
    Survey = 'Everest'
    Survey = Survey.upper()

    # Global path to where the spectra are held
    if Survey in ['EVEREST']:
        redux=f'/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative'

    else:
        redux='/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative'

    #Extinction - Setting here only applies to DESI
    extinction_model = F99(Rv=3.1)
    
    TableID = 'MainTest'
    BLOCK_ID = 'Everest_Main_Dark_1_Test' #DESI Only

########################
# General Configuration
########################

debug = False
header_print = False
ignore_score_check = True  # disables the check that the candidate score matches the one stored in the database
Follow_Up = True

c = constants.value('speed of light in vacuum') / 1000  # Speed of light in kms^-1

config_parameters = Hirogen_Functions.main_config()  # Draws from centralised parameter declarations

Lower_Wave = config_parameters[0]
Upper_Wave = config_parameters[1]

Lower_Shift = config_parameters[2]
Upper_Shift = config_parameters[3]

# Lower_Shift = -5000
# Upper_Shift = 5000

# Any object with a candidate score equal to or exceeding this value will be treated as an ECLE candidate
Candidate_Threshold = 7 #config_parameters[4]

# Maximum possible candidate score
Candidate_Score_Max = config_parameters[5]

# To qualify as a line detection for candidate selection the overall feature pEQW must exceed this value:
LineDetection_pEQW_Threshold = config_parameters[6]
LineDetection_Max_Threshold = config_parameters[7]

LineDetection_Max_Above_Av_Continua_Threshold = config_parameters[8]

# To qualify as a line detection for candidate selection the maximum flux point of the feature must occur
# within this +_ kms^-1 of the zero velocity point
LineDetection_Peak_Tolerance = config_parameters[9]
Line_Peak_Location_Region = config_parameters[10]

Line_Peak_Region_Minima_Threshold = config_parameters[11]

Strong_EQW_Threshold = config_parameters[12]
Strong_Peak_Max_Threshold = config_parameters[13]

# Produces an ascii file report of the scoring for each line
Line_Report = True

########################
# Smoothing and Filtering Configuration
########################

Spectres_Rebin = False
Rebin_Res = 5

# Applies a boxcar smoothing function of width to the spectrum on read in - SDSS Only
Default_Smoothing = True

#######
# DESI Specific
#######

FFT_Denoise = False  # Applies a FFT to reduce the erratic noise on each pixel
FFT_Threshold_Str = '1e5' # Unfortunate side effect
FFT_Threshold = 1e5

Gaussian_Smooth = False # Controls if a gaussian smoothing factor is used
Gaussian_Sigma = 3

Median_Filter = False # Defaults for DESI spectra: No boxcar smoothing but has a median filter applied
Median_Filter_Kernel = 3

Settings_Config = f'{int(Spectres_Rebin)}_{int(Rebin_Res)}_{int(Default_Smoothing)}_{int(FFT_Denoise)}_{FFT_Threshold_Str}_{int(Gaussian_Smooth)}_{Gaussian_Sigma}_{int(Median_Filter)}_{Median_Filter_Kernel}'
# Rolls all the smoothing / denoise configuration settings into one string for storage and comparison

########################
# Plot configuration
########################
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 30

left, width = 0.07, 0.65
bottom, height = 0.1, .8
box = [left, bottom, width, height]


# VelocityTickSpacing = 300 # This was used to set the tick spacing on the snapshot plots but its very slow

print("Import and configuration complete")

In [ ]:
# Functions

def snap_shot_figure(object_name,
                     object_id,
                     object_spectrum,
                     database_data,
                     lines_to_plot,
                     line_labels,
                     line_data,
                     lines_for_scoring,
                     lick_delta_index,
                     lick_delta_index_err,
                     score,
                     save_path,
                     text_flag=True
                     # This flag controls the inclusion of a text 'figure' at the bottom with descriptive info
                     ):

    Wavelength = object_spectrum[0]
    Flux = object_spectrum[1]

    recorded_score = database_data[11]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]

    if text_flag:
        Snapshot_Fig = plt.figure(figsize=(40, 28))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(4, 4, hspace=0.05, wspace=0.01)

    else:
        Snapshot_Fig = plt.figure(figsize=(40, 21))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(3, 4, hspace=0.05, wspace=0.01)

    Full_Spec_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[0, 0:4])
    Full_Spec_Ax.set_xlabel(r"Rest Wavelength ($\AA$)")
    Full_Spec_Ax.set_ylabel(r"Flux (10$^{-17}$ erg/cm$^{2}$/s/$\AA$)")
    Full_Spec_Ax.step(Wavelength, Flux, color='black', linewidth=3, zorder=100)

    try:
        Lower_Spec_Wave_Limit = Hirogen_Functions.round_down_to_nearest(np.nanmin(np.array(Wavelength)), 250)
    except ValueError:
        Lower_Spec_Wave_Limit = 0

    try:
        Upper_Spec_Wave_Limit = Hirogen_Functions.round_up_to_nearest(np.nanmax(np.array(Wavelength)), 250)
    except ValueError:
        Upper_Spec_Wave_Limit = 10000

    if object_name == "SDSS J0748+4712 MMT":
        Full_Spec_Ax.set_ylim(ymin=0, ymax=40)

    spec_ax_y_lims = Full_Spec_Ax.get_ylim()

    plt.annotate(
        f'Object: {object_name}',
        (((Upper_Spec_Wave_Limit - Lower_Spec_Wave_Limit) / 2) + Lower_Spec_Wave_Limit, spec_ax_y_lims[1] * 0.9),
        ha='center',
        va='center', zorder=200, color='k', backgroundcolor='1.0', fontsize=32)

    if object_name in ["SDSS J0748+4712", "SDSS J0748+4712 MMT"]:
        plt.axvspan(xmin=4300, xmax=4900, color='0.9')

    Full_Spec_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    Spec_Ticks = []

    x = 0
    y = 1

    for ii, item in enumerate(lines_to_plot):

        Current_Line_For_Plotting_Data = line_data[lines_to_plot[ii]]

        Full_Spec_Ax.axvline(x=Current_Line_For_Plotting_Data[0], color=Current_Line_For_Plotting_Data[1],
                             linestyle='-', linewidth=4)
        Spec_Ticks.append(Current_Line_For_Plotting_Data[0], )

        Feature_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[y, x])
        Feature_Ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

        """
        # Overwrite the xaxis limits for certain lines if needed
        if lines_to_plot[ii] == '[NII]6548':
            lower_shift = -3500
            upper_shift = 3500

        else:
            lower_shift = Lower_Shift
            upper_shift = Upper_Shift
        """

        Feature_Ax.set_xlim(Lower_Shift, Upper_Shift)
        # Feature_Ax.xaxis.set_major_locator(ticker.MultipleLocator(VelocityTickSpacing))  #Quite slow

        Shift_to_plot = []
        Flux_to_plot = []

        try:

            for jj, point in enumerate(line_data[item][3]):

                if Lower_Shift - 10 < point < Upper_Shift + 10:
                    Shift_to_plot.append(point)
                    Flux_to_plot.append(line_data[item][7][jj])

        except TypeError:
            Shift_to_plot = []
            Flux_to_plot=[]

        plt.axhline(y=1, color='k', linestyle='-')
        plt.axvline(x=0, color=Current_Line_For_Plotting_Data[1], linestyle='-', linewidth=4)

        plt.axhline(
            y=Current_Line_For_Plotting_Data[9] * LineDetection_Max_Above_Av_Continua_Threshold,
            linestyle='--', color='k')

        try:
            plt.axvline(x=line_data[lines_to_plot[ii]][12][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][13][1], color='orange', linestyle='--', linewidth=3)

            plt.axvline(x=line_data[lines_to_plot[ii]][14][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][15][1], color='orange', linestyle='--', linewidth=3)
        except TypeError:
            print('Unable to add continuum measurement lines: '
                  'Likely spectral gap or line is not covered by the spectrum due to redshift')

        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] - 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)
        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] + 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)

        plt.axvline(x=LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)
        plt.axvline(x=-1 * LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)
        plt.axhline(y=-0.3, color='k', linestyle=':', linewidth=3)

        plt.step(Shift_to_plot, Flux_to_plot, color='k', linewidth=3)

        try:
            Rounded_Line_Max = Hirogen_Functions.round_up_to_nearest(np.nanmax(Flux_to_plot), 0.05) + 0.01
        except ValueError:
            print("Calculation of line max has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Max = 2

        try:
            Rounded_Line_Min = Hirogen_Functions.round_down_to_nearest(np.nanmin(Flux_to_plot), 0.05) + 0.01
        except ValueError:
            print("Calculation of line min has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Min = -0.5

        plt.ylim(ymax=Rounded_Line_Max, ymin=Rounded_Line_Min)

        plt.annotate(
            text=f'pEQW: {line_data[item][8]:.2f}\nLine Flux: {line_data[item][16]:.2f}',xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='right',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        plt.annotate(
            f'{line_labels[ii]}',
            xy=(-2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='left',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        if lines_to_plot[ii] == 'Hdelta':

            plt.annotate(
                text=f'Lick H$\delta_A$: {lick_delta_index:.2f}\u00B1{lick_delta_index_err:.2f}',
                xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 0.5 + Rounded_Line_Min), ha='right',
                va='center',
                zorder=10, color='k', backgroundcolor='1.0')

        if y == 2:
            Feature_Ax.set_xlabel(r"Relative Velocity (km/s)")
            Feature_Ax.tick_params('x', labelrotation=30, width=3, length=7)
        else:
            Feature_Ax.tick_params(labelbottom=False, bottom=True, width=3, length=7)

        if x == 0:
            Feature_Ax.set_ylabel("Scaled Flux")

        x = x + 1

        if x == 4:
            x = x - 4
            y = y + 1

    if text_flag:

        InfoSubplot = Snapshot_Fig.add_subplot(Snapshot_Grid[y, 0:4])
        InfoSubplot.axis('off')

        plt.xlim(0, 1)

        plt.axhline(y=0.95, color='k')
        plt.annotate(text=f"Candidate Score: {score} / {Candidate_Score_Max}", xy=(0.01, 0.8), ha='left', va='bottom')
        plt.annotate(text=f"Recorded Candidate Score: {recorded_score} / {Candidate_Score_Max}", xy=(0.01, 0.7),
                     ha='left', va='bottom')
        plt.annotate(text=f"Candidate Threshold: {Candidate_Threshold}", xy=(0.01, 0.6), ha='left', va='bottom')

        plt.annotate(text=f"Strong FeVII: {bool(fe_vii_flag)}",
                     xy=(0.01, 0.5), ha='left', va='bottom')
        plt.annotate(text=f"Strong FeXIV: {bool(fe_xiv_flag)}",
                     xy=(0.01, 0.4), ha='left', va='bottom')

        plt.annotate(text="Scored Line Detections:",
                     xy=(0.25, 0.8), ha='left', va='bottom')

        plt.annotate(text="Regions used for continuum fit indicated by the vertical dashed orange lines", xy=(0.5, 0.8),
                     ha='left', va='bottom')
        plt.annotate(text="Region included in the pEQW calculation indicated by the vertical dash-dot brown lines",
                     xy=(0.5, 0.7),
                     ha='left', va='bottom')
        plt.annotate(
            text="Dashed horizontal black line displays the scaled flux threshold required for a line detection",
            xy=(0.5, 0.6),
            ha='left', va='bottom')
        plt.annotate(text="Solid horizontal black line indicates the scaled continuum flux level for clarity",
                     xy=(0.5, 0.5),
                     ha='left', va='bottom')

        if object_name == "SDSS J0748+5712":
            plt.annotate(text="Shaded region indicates the location of a broad HeII feature linked to the TDE",
                         xy=(0.5, 0.4),
                         ha='left', va='bottom')

        y_annotate = 0.65
        for ii, item in enumerate(lines_to_plot):

            if lines_to_plot[ii] in lines_for_scoring:
                plt.annotate(
                    text=f"{lines_to_plot[ii]}: "
                         f"{line_data[item][11]}",
                    xy=(0.25, y_annotate), ha='left', va='bottom')
                y_annotate -= 0.1

    # Snapshot_Grid.tight_layout(Snapshot_Fig)

    # Adding the tick labels to the full spectrum
    Spec_Label_Ax = Full_Spec_Ax.twiny()

    Spec_Label_Ax.set_xticks(Spec_Ticks)
    Spec_Label_Ax.set_xticklabels(line_labels, rotation=15)
    Spec_Label_Ax.tick_params(top=False, bottom=False, left=False, right=False)
    Spec_Label_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        Snapshot_Fig.savefig(
            f"{save_path}/Primary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
            dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Primary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Primary_Spec_Snapshot_Plot_for_{object_name}_StrongFeXIV.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)
    else:
        Snapshot_Fig.savefig(f"{save_path}/Primary_Spec_Snapshot_Plot_for_{object_name}.pdf", dpi=200,
                             bbox_inches='tight', transparent = False)
    #Snapshot_Fig.savefig(
    #f"FFT_Test/Primary_Spec_Snapshot_Plot_for_{object_name}_FFT_Threshold_thresold_1e5_Median_Filter_Kernel_3.pdf",
    #dpi=200, bbox_inches='tight', transparent = False)

    Full_Spec_Ax.clear()
    # Feature_Ax.clear()

    plt.figure().clear()
    plt.close(Snapshot_Fig)
    return None


def presentation_snap_shot_figure(object_name,
                     object_id,
                     object_spectrum,
                     database_data,
                     lines_to_plot,
                     line_labels,
                     line_data,
                     lines_for_scoring,
                     lick_delta_index,
                     lick_delta_index_err,
                     score,
                     save_path,
                     zoom_lines,
                     zoom_line_labels
                     # This flag controls the inclusion of a text 'figure' at the bottom with descriptive info
                     ):

    Wavelength = object_spectrum[0]
    Flux = object_spectrum[1]

    recorded_score = database_data[11]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]

    Snapshot_Fig = plt.figure(figsize=(40, 20))
    Snapshot_Grid = Snapshot_Fig.add_gridspec(2, 4, hspace=0.05, wspace=0)

    Full_Spec_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[0, 0:4])
    Full_Spec_Ax.set_xlabel(r"Rest Wavelength ($\AA$)", fontsize = 36)
    #Full_Spec_Ax.set_ylabel(r"Flux (10$^{-17}$ erg/cm$^{2}$/s/$\AA$)")
    Full_Spec_Ax.set_ylabel(r"Flux", fontsize=36)
    plt.yticks([])
    Full_Spec_Ax.step(Wavelength, Flux, color='black', linewidth=3, zorder=100)

    Lower_Spec_Wave_Limit = 4000
    Upper_Spec_Wave_Limit = 8000

    Upper_Scaled_Flux_Limit = Hirogen_Functions.round_up_to_nearest(np.nanmax(np.array(Flux)), 5)
    Full_Spec_Ax.set_ylim(0, Upper_Scaled_Flux_Limit)

    if object_name == "SDSS J0748+4712 MMT":
        Full_Spec_Ax.set_ylim(ymin=0, ymax=40)

    spec_ax_y_lims = Full_Spec_Ax.get_ylim()

    plt.annotate(
        f'{ObjectName}',
        (((Upper_Spec_Wave_Limit - Lower_Spec_Wave_Limit) / 2) + Lower_Spec_Wave_Limit, spec_ax_y_lims[1] * 0.8),
        ha='center',
        va='center', zorder=200, color='k', backgroundcolor='1.0', fontsize=36)

    if object_name in ["SDSS J0748+4712", "SDSS J0748+4712 MMT"]:
        plt.axvspan(xmin=4300, xmax=4900, color='0.9')

    Full_Spec_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)
    #Full_Spec_Ax.tick_params('y',labelleft=False)

    Spec_Ticks = []

    x = 0
    y = 1

    for ii, item in enumerate(lines_to_plot):
        Current_Line_For_Plotting_Data = line_data[lines_to_plot[ii]]
        Full_Spec_Ax.axvline(x=Current_Line_For_Plotting_Data[0], color=Current_Line_For_Plotting_Data[1],
                             linestyle='-', linewidth=4)
        Spec_Ticks.append(Current_Line_For_Plotting_Data[0], )

    for ii, item in enumerate(zoom_lines):

        Current_Line_For_Plotting_Data = line_data[zoom_lines[ii]]

        Feature_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[y, x])
        #Feature_Ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        plt.yticks([])

        """
        # Overwrite the xaxis limits for certain lines if needed
        if lines_to_plot[ii] == '[NII]6548':
            lower_shift = -3500
            upper_shift = 3500

        else:
            lower_shift = Lower_Shift
            upper_shift = Upper_Shift
        """

        Feature_Ax.set_xlim(Lower_Shift, Upper_Shift)
        # Feature_Ax.xaxis.set_major_locator(ticker.MultipleLocator(VelocityTickSpacing))  #Quite slow

        Shift_to_plot = []
        Flux_to_plot = []

        try:

            for jj, point in enumerate(line_data[item][3]):

                if Lower_Shift - 10 < point < Upper_Shift + 10:
                    Shift_to_plot.append(point)
                    Flux_to_plot.append(line_data[item][7][jj])

        except TypeError:
            Shift_to_plot = []
            Flux_to_plot=[]

        plt.axhline(y=1, color='k', linestyle='-')
        plt.axvline(x=0, color=Current_Line_For_Plotting_Data[1], linestyle='-', linewidth=4)

        plt.step(Shift_to_plot, Flux_to_plot, color='k', linewidth=3)

        try:
            Rounded_Line_Max = Hirogen_Functions.round_up_to_nearest(np.nanmax(Flux_to_plot), 0.05) + 0.01
        except ValueError:
            print("Calculation of line max has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Max = 2

        try:
            Rounded_Line_Min = Hirogen_Functions.round_down_to_nearest(np.nanmin(Flux_to_plot), 0.05) + 0.01
        except ValueError:
            print("Calculation of line min has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Min = -0.5

        plt.ylim(ymax=Rounded_Line_Max, ymin=Rounded_Line_Min)

        """
        plt.annotate(
            text=f'pEQW: {line_data[item][8]:.2f}\nLine Flux: {line_data[item][16]:.2f}',xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='right',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')
        """

        plt.annotate(
            f'{zoom_line_labels[ii]}',
            xy=(-2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.7 + Rounded_Line_Min), ha='left',
            va='center', fontsize = 32
            ,
            zorder=10, color='k', backgroundcolor='1.0')


        if lines_to_plot[ii] == 'Hdelta':
            plt.annotate(
                text=f'Lick H$\delta_A$: {lick_delta_index:.2f}\u00B1{lick_delta_index_err:.2f}',
                xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.6 + Rounded_Line_Min), ha='right',
                va='center',
                zorder=10, color='k', backgroundcolor='1.0')
    
        if y == 1:
            Feature_Ax.set_xlabel(r"Relative Velocity (km/s)")
            Feature_Ax.tick_params('x',labelrotation=30, width=3, length=7)

        if x == 0:
            Feature_Ax.set_ylabel("Scaled Flux", fontsize = 36)

        x = x + 1

        if x == 4:
            x = x - 4
            y = y + 1

    # Adding the tick labels to the full spectrum
    Spec_Label_Ax = Full_Spec_Ax.twiny()

    Spec_Label_Ax.set_xticks(Spec_Ticks)
    Spec_Label_Ax.set_xticklabels(line_labels, rotation=25, fontsize=36)
    Spec_Label_Ax.tick_params(top=False, bottom=False, left=False, right=False)
    Spec_Label_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    Snapshot_Fig.savefig(
    f"{save_path}/Presentation_Plot_for_{object_name}.pdf",
    dpi=200, bbox_inches='tight', transparent = False)

    Full_Spec_Ax.clear()
    # Feature_Ax.clear()

    plt.figure().clear()
    plt.close(Snapshot_Fig)
    return None


def snap_shot_figure_spec_only(object_name,
                               object_id,
                               object_spectrum,
                               database_data,
                               lines_to_plot,
                               line_labels,
                               line_data,
                               lines_for_scoring,
                               lick_delta_index,
                               lick_delta_index_err,
                               score,
                               save_path,
                               text_flag=True
                               # This flag controls the inclusion of a text 'figure' at the bottom with descriptive info
                               ):

    Wavelength = object_spectrum[0]
    Flux = object_spectrum[1]

    recorded_score = database_data[11]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]

    if text_flag:
        Snapshot_Fig = plt.figure(figsize=(40, 38))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(4, 4, hspace=0.05, wspace=0.01)

    else:
        Snapshot_Fig = plt.figure(figsize=(28, 12))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(3, 1, hspace=0.05, wspace=0.01)

    Full_Spec_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[0, 0:4])
    Full_Spec_Ax.set_xlabel(r"Rest Wavelength ($\AA$)")
    Full_Spec_Ax.set_ylabel(r"Flux (10$^{-17}$ erg/cm$^{2}$/s/$\AA$)")
    Full_Spec_Ax.step(Wavelength, Flux, color='black', linewidth=3, zorder=100)

    try:
        Lower_Spec_Wave_Limit = Hirogen_Functions.round_down_to_nearest(np.min(np.array(Wavelength)), 250)
    except ValueError:
        Lower_Spec_Wave_Limit = 0

    try:
        Upper_Spec_Wave_Limit = Hirogen_Functions.round_up_to_nearest(np.max(np.array(Wavelength)), 250)
    except ValueError:
        Upper_Spec_Wave_Limit = 10000

    spec_ax_y_lims = Full_Spec_Ax.get_ylim()

    plt.annotate(
        f'Object: {object_name}',
        (((Upper_Spec_Wave_Limit - Lower_Spec_Wave_Limit) / 2) + Lower_Spec_Wave_Limit, spec_ax_y_lims[1] * 0.9),
        ha='center',
        va='center', zorder=200, color='k', backgroundcolor='1.0', fontsize=32)

    if object_name in ["SDSS J0748+4712", "SDSS J0748+4712 MMT"]:
        plt.axvspan(xmin=4300, xmax=4900, color='0.9')

    Full_Spec_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    if object_name == "SDSS J0748+4712 MMT":
        Full_Spec_Ax.set_ylim(ymin=0.5e-17, ymax=30e-17)

    Spec_Ticks = []

    x = 0
    y = 1

    for ii, item in enumerate(lines_to_plot):
        Current_Line_For_Plotting_Data = line_data[lines_to_plot[ii]]
        Spec_Ticks.append(Current_Line_For_Plotting_Data[0])
        Full_Spec_Ax.axvline(x=Current_Line_For_Plotting_Data[0], color=Current_Line_For_Plotting_Data[1],
                             linestyle='-', linewidth=4)

    # Adding the tick labels to the full spectrum
    Spec_Label_Ax = Full_Spec_Ax.twiny()

    Spec_Label_Ax.set_xticks(Spec_Ticks)
    Spec_Label_Ax.set_xticklabels(line_labels, rotation=15)
    Spec_Label_Ax.tick_params(top=False, bottom=False, left=False, right=False)
    Spec_Label_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Spectrum_Only_{object_name}_StrongFeVII_StrongFeXIV.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Spectrum_Only_{object_name}_StrongFeVII.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Spectrum_Only_{object_name}_StrongFeXIV.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)
    else:
        Snapshot_Fig.savefig(f"{save_path}/Spectrum_Only_{object_name}.pdf", dpi=200, bbox_inches='tight', transparent = False)

    Full_Spec_Ax.clear()
    # Feature_Ax.clear()

    plt.figure().clear()
    plt.close(Snapshot_Fig)
    return None


def secondary_snap_shot_figure(object_name,
                               object_id,
                               object_spectrum,
                               database_data,
                               lines_to_plot,
                               line_labels,
                               line_data,
                               score,
                               save_path,
                               text_flag=True
                               # This flag controls the inclusion of a text 'figure' at the bottom with descriptive info
                               ):

    Wavelength = object_spectrum[0]
    Flux = object_spectrum[1]

    recorded_score = database_data[11]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]
 
    if text_flag:
        Snapshot_Fig = plt.figure(figsize=(40, 42))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(7, 4, hspace=0.05, wspace=0.01)

    else:
        Snapshot_Fig = plt.figure(figsize=(40, 35))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(6, 4, hspace=0.05, wspace=0.01)

    Full_Spec_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[0, 0:4])
    Full_Spec_Ax.set_xlabel(r"Rest Wavelength ($\AA$)")
    Full_Spec_Ax.set_ylabel(r"Flux (10$^{-17}$ erg/cm$^{2}$/s/$\AA$)")
    Full_Spec_Ax.step(Wavelength, Flux, color='black', linewidth=3, zorder=100)

    try:
        Lower_Spec_Wave_Limit = Hirogen_Functions.round_down_to_nearest(np.min(np.array(Wavelength)), 250)
    except ValueError:
        Lower_Spec_Wave_Limit = 0

    try:
        Upper_Spec_Wave_Limit = Hirogen_Functions.round_up_to_nearest(np.max(np.array(Wavelength)), 250)
    except ValueError:
        Upper_Spec_Wave_Limit = 10000

    spec_ax_y_lims = Full_Spec_Ax.get_ylim()

    plt.annotate(
        f'Object: {object_name}',
        (((Upper_Spec_Wave_Limit - Lower_Spec_Wave_Limit) / 2) + Lower_Spec_Wave_Limit, spec_ax_y_lims[1] * 0.9),
        ha='center',
        va='center', zorder=200, color='k', backgroundcolor='1.0', fontsize=32)

    if object_name == "SDSS J0748+5712":
        plt.axvspan(xmin=4250, xmax=4900, color='0.9')

    Full_Spec_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    Spec_Ticks = []

    x = 0
    y = 1

    for ii, item in enumerate(lines_to_plot):

        Current_Line_For_Plotting_Data = line_data[lines_to_plot[ii]]

        Full_Spec_Ax.axvline(x=Current_Line_For_Plotting_Data[0], color=Current_Line_For_Plotting_Data[1],
                             linestyle='-', linewidth=3)
        Spec_Ticks.append(Current_Line_For_Plotting_Data[0], )

        Feature_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[y, x])

        """
        # Overwrite the xaxis limits for certain lines if needed
        if lines_to_plot[ii] == '[NII]6548':
            lower_shift = -3500
            upper_shift = 3500

        else:
            lower_shift = Lower_Shift
            upper_shift = Upper_Shift
        """

        Feature_Ax.set_xlim(Lower_Shift, Upper_Shift)

        # Feature_Ax.xaxis.set_major_locator(ticker.MultipleLocator(VelocityTickSpacing))  #Quite slow

        plt.axhline(y=1, color='k', linestyle='-')
        plt.axvline(x=0, color=Current_Line_For_Plotting_Data[1], linestyle='-', linewidth=3)

        plt.axhline(
            y=Current_Line_For_Plotting_Data[9] * LineDetection_Max_Above_Av_Continua_Threshold,
            linestyle='--', color='k')

        try:
            plt.axvline(x=line_data[lines_to_plot[ii]][12][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][13][1], color='orange', linestyle='--', linewidth=3)

            plt.axvline(x=line_data[lines_to_plot[ii]][14][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][15][1], color='orange', linestyle='--', linewidth=3)
        except TypeError:
            print('Unable to add continuum measurement lines: '
                  'Likely spectral gap or line is not covered by the spectrum due to redshift')

        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] - 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)
        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] + 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)

        plt.axvline(x=LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)
        plt.axvline(x=-1 * LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)

        plt.step(line_data[item][3], line_data[item][7], color='k', linewidth=3)

        try:
            Rounded_Line_Max = Hirogen_Functions.round_up_to_nearest(np.nanmax(line_data[item][7]), 0.05) + 0.01
        except ValueError:
            print("Calculation of line max has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Max = 2

        try:
            Rounded_Line_Min = Hirogen_Functions.round_down_to_nearest(np.nanmin(line_data[item][7]), 0.05) + 0.01
        except ValueError:
            print("Calculation of line min has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Min = -0.5

        plt.ylim(ymax=Rounded_Line_Max, ymin=Rounded_Line_Min)

        plt.annotate(
            text=f'pEQW: {line_data[item][8]:.2f}\nLine Flux: {line_data[item][16]:.2f}',xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='right',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        plt.annotate(
            text=f'{line_labels[ii]}',xy=(-2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='left',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        Feature_Ax.set_xlabel(r"Relative Velocity (km/s)")
        Feature_Ax.tick_params('x', labelrotation=30)

        if x == 0:
            Feature_Ax.set_ylabel("Scaled Flux")

        x = x + 1

        if x == 4:
            x = x - 4
            y = y + 1

    if text_flag:
        y = y + 1  # This line is only needed when the number of line plots isn't a full row
        InfoSubplot = Snapshot_Fig.add_subplot(Snapshot_Grid[y, 0:4])
        InfoSubplot.axis('off')

        plt.xlim(0, 1)

        plt.axhline(y=0.95, color='k')
        plt.annotate(text=f"Candidate Score: {score} / {Candidate_Score_Max}", xy=(0.01, 0.8), ha='left', va='bottom')
        plt.annotate(text=f"Candidate Threshold: {Candidate_Threshold}", xy=(0.01, 0.7), ha='left', va='bottom')

        plt.annotate(text=f"Strong FeVII: {bool(fe_vii_flag)}",
                     xy=(0.01, 0.5), ha='left', va='bottom')
        plt.annotate(text=f"Strong FeXIV: {bool(fe_xiv_flag)}",
                     xy=(0.01, 0.4), ha='left', va='bottom')

        plt.annotate(text="Regions used for continuum fit indicated by the vertical dashed orange lines", xy=(0.5, 0.8),
                     ha='left', va='bottom')
        plt.annotate(text="Region included in the pEQW calculation indicated by the vertical dash-dot brown lines",
                     xy=(0.5, 0.7),
                     ha='left', va='bottom')
        plt.annotate(
            text="Dashed horizontal black line displays the scaled flux threshold required for a line detection",
            xy=(0.5, 0.6),
            ha='left', va='bottom')
        plt.annotate(text="Solid horizontal black line indicates the scaled continuum flux level for clarity",
                     xy=(0.5, 0.5),
                     ha='left', va='bottom')

        if object_name == "SDSS J0748+5712":
            plt.annotate(text="Shaded region indicates the location of a broad HeII feature linked to the TDE",
                         xy=(0.5, 0.4),
                         ha='left', va='bottom')

    # Snapshot_Grid.tight_layout(Snapshot_Fig)

    # Adding the tick labels to the full spectrum
    Spec_Label_Ax = Full_Spec_Ax.twiny()

    Spec_Label_Ax.set_xticks(Spec_Ticks)
    Spec_Label_Ax.set_xticklabels(line_labels, rotation=15)
    Spec_Label_Ax.tick_params(top=False, bottom=False, left=False, right=False)
    Spec_Label_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        Snapshot_Fig.savefig(
            f"{save_path}/Secondary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
            dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Secondary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Secondary_Spec_Snapshot_Plot_for_{object_name}_StrongFeXIV.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)
    else:
        Snapshot_Fig.savefig(f"{save_path}/Secondary_Spec_Snapshot_Plot_for_{object_name}.pdf", dpi=200,
                             bbox_inches='tight', transparent = False)

    Full_Spec_Ax.clear()
    # Feature_Ax.clear()

    plt.figure().clear()
    plt.close(Snapshot_Fig)
    return None


def tertiary_snap_shot_figure(object_name,
                              object_id,
                              object_spectrum,
                              database_data,
                              lines_to_plot,
                              line_labels,
                              line_data,
                              score,
                              save_path,
                              text_flag=True
                              # This flag controls the inclusion of a text 'figure' at the bottom with descriptive info
                              ):

    Wavelength = object_spectrum[0]
    Flux = object_spectrum[1]

    recorded_score = database_data[11]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]

    if text_flag:
        Snapshot_Fig = plt.figure(figsize=(40, 28))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(4, 4, hspace=0.05, wspace=0.01)

    else:
        Snapshot_Fig = plt.figure(figsize=(40, 21))
        Snapshot_Grid = Snapshot_Fig.add_gridspec(3, 4, hspace=0.05, wspace=0.01)

    Full_Spec_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[0, 0:4])
    Full_Spec_Ax.set_xlabel(r"Rest Wavelength ($\AA$)")
    Full_Spec_Ax.set_ylabel(r"Flux (10$^{-17}$ erg/cm$^{2}$/s/$\AA$)")
    Full_Spec_Ax.step(Wavelength, Flux, color='black', linewidth=3, zorder=100)

    try:
        Lower_Spec_Wave_Limit = Hirogen_Functions.round_down_to_nearest(np.min(np.array(Wavelength)), 250)
    except ValueError:
        Lower_Spec_Wave_Limit = 0

    try:
        Upper_Spec_Wave_Limit = Hirogen_Functions.round_up_to_nearest(np.max(np.array(Wavelength)), 250)
    except ValueError:
        Upper_Spec_Wave_Limit = 10000

    spec_ax_y_lims = Full_Spec_Ax.get_ylim()

    plt.annotate(
        f'Object: {object_name}',
        (((Upper_Spec_Wave_Limit - Lower_Spec_Wave_Limit) / 2) + Lower_Spec_Wave_Limit, spec_ax_y_lims[1] * 0.9),
        ha='center',
        va='center', zorder=200, color='k', backgroundcolor='1.0', fontsize=32)

    if object_name == "SDSS J0748+5712":
        plt.axvspan(xmin=4250, xmax=4900, color='0.9')

    Full_Spec_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    Spec_Ticks = []

    x = 0
    y = 1

    for ii, item in enumerate(lines_to_plot):

        Current_Line_For_Plotting_Data = line_data[lines_to_plot[ii]]

        Full_Spec_Ax.axvline(x=Current_Line_For_Plotting_Data[0], color=Current_Line_For_Plotting_Data[1],
                             linestyle='-', linewidth=3)
        Spec_Ticks.append(Current_Line_For_Plotting_Data[0], )

        Feature_Ax = Snapshot_Fig.add_subplot(Snapshot_Grid[y, x])

        """
        # Overwrite the xaxis limits for certain lines if needed
        if lines_to_plot[ii] == '[NII]6548':
            lower_shift = -3500
            upper_shift = 3500

        else:
            lower_shift = Lower_Shift
            upper_shift = Upper_Shift
        """

        Feature_Ax.set_xlim(Lower_Shift, Upper_Shift)

        # Feature_Ax.xaxis.set_major_locator(ticker.MultipleLocator(VelocityTickSpacing))  #Quite slow

        plt.axhline(y=1, color='k', linestyle='-')
        plt.axvline(x=0, color=Current_Line_For_Plotting_Data[1], linestyle='-', linewidth=3)

        plt.axhline(
            y=Current_Line_For_Plotting_Data[11] * LineDetection_Max_Above_Av_Continua_Threshold,
            linestyle='--', color='k')

        try:
            plt.axvline(x=line_data[lines_to_plot[ii]][12][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][13][1], color='orange', linestyle='--', linewidth=3)

            plt.axvline(x=line_data[lines_to_plot[ii]][14][1], color='orange', linestyle='--', linewidth=3)
            plt.axvline(x=line_data[lines_to_plot[ii]][15][1], color='orange', linestyle='--', linewidth=3)
        except TypeError:
            print('Unable to add continuum measurement lines: '
                  'Likely spectral gap or line is not covered by the spectrum due to redshift')

        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] - 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)
        plt.axvline(x=((((line_data[lines_to_plot[ii]][0] + 12) * c) / line_data[lines_to_plot[ii]][0]) - c),
                    color='brown', linestyle='-.', linewidth=3)

        plt.axvline(x=LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)
        plt.axvline(x=-1 * LineDetection_Peak_Tolerance, color='magenta', linestyle=':', linewidth=3)

        plt.step(line_data[item][3], line_data[item][7], color='k', linewidth=3)

        try:
            Rounded_Line_Max = Hirogen_Functions.round_up_to_nearest(np.nanmax(line_data[item][7]), 0.05) + 0.01
        except ValueError:
            print("Calculation of line max has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Max = 2

        try:
            Rounded_Line_Min = Hirogen_Functions.round_down_to_nearest(np.nanmin(line_data[item][7]), 0.05) + 0.01
        except ValueError:
            print("Calculation of line min has failed: Switching to default"
                  "\nLikely cause is a gap in the spectrum")
            Rounded_Line_Min = -0.5

        plt.ylim(ymax=Rounded_Line_Max, ymin=Rounded_Line_Min)

        plt.annotate(
            text=f'pEQW: {line_data[item][8]:.2f}\nLine Flux: {line_data[item][16]:.2f}',xy=(2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='right',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        plt.annotate(
            text=f'{line_labels[ii]}',xy=(-2900, float(((Rounded_Line_Max - Rounded_Line_Min) / 2)) * 1.8 + Rounded_Line_Min), ha='left',
            va='center',
            zorder=10, color='k', backgroundcolor='1.0')

        Feature_Ax.set_xlabel(r"Relative Velocity (km/s)")
        Feature_Ax.tick_params('x', labelrotation=30)

        if x == 0:
            Feature_Ax.set_ylabel("Scaled Flux")

        x = x + 1

        if x == 4:
            x = x - 4
            y = y + 1

    if text_flag:

        y = y + 1

        InfoSubplot = Snapshot_Fig.add_subplot(Snapshot_Grid[y, 0:4])
        InfoSubplot.axis('off')

        plt.xlim(0, 1)

        plt.axhline(y=0.95, color='k')
        plt.annotate(text=f"Candidate Score: {score} / {Candidate_Score_Max}", xy=(0.01, 0.8), ha='left', va='bottom')
        plt.annotate(text=f"Candidate Threshold: {Candidate_Threshold}", xy=(0.01, 0.7), ha='left', va='bottom')

        plt.annotate(text=f"Strong FeVII: {bool(fe_vii_flag)}",
                     xy=(0.01, 0.5), ha='left', va='bottom')
        plt.annotate(text=f"Strong FeXIV: {bool(fe_xiv_flag)}",
                     xy=(0.01, 0.4), ha='left', va='bottom')

        plt.annotate(text="Regions used for continuum fit indicated by the vertical dashed orange lines", xy=(0.5, 0.8),
                     ha='left', va='bottom')
        plt.annotate(text="Region included in the pEQW calculation indicated by the vertical dash-dot brown lines",
                     xy=(0.5, 0.7),
                     ha='left', va='bottom')
        plt.annotate(
            text="Dashed horizontal black line displays the scaled flux threshold required for a line detection",
            xy=(0.5, 0.6),
            ha='left', va='bottom')
        plt.annotate(text="Solid horizontal black line indicates the scaled continuum flux level for clarity",
                     xy=(0.5, 0.5),
                     ha='left', va='bottom')

        if object_name == "SDSS J0748+5712":
            plt.annotate(text="Shaded region indicates the location of a broad HeII feature linked to the TDE",
                         xy=(0.5, 0.4),
                         ha='left', va='bottom')

    # Snapshot_Grid.tight_layout(Snapshot_Fig)

    # Adding the tick labels to the full spectrum
    Spec_Label_Ax = Full_Spec_Ax.twiny()

    Spec_Label_Ax.set_xticks(Spec_Ticks)
    Spec_Label_Ax.set_xticklabels(line_labels, rotation=15)
    Spec_Label_Ax.tick_params(top=False, bottom=False, left=False, right=False)
    Spec_Label_Ax.set_xlim(xmin=Lower_Spec_Wave_Limit, xmax=Upper_Spec_Wave_Limit)

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        Snapshot_Fig.savefig(
            f"{save_path}/Tertiary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
            dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Tertiary_Spec_Snapshot_Plot_for_{object_name}_StrongFeVII.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        Snapshot_Fig.savefig(f"{save_path}/Tertiary_Spec_Snapshot_Plot_for_{object_name}_StrongFeXIV.pdf",
                             dpi=200, bbox_inches='tight', transparent = False)
    else:
        Snapshot_Fig.savefig(f"{save_path}/Tertiary_Spec_Snapshot_Plot_for_{object_name}.pdf", dpi=200,
                             bbox_inches='tight', transparent = False)

    Full_Spec_Ax.clear()
    # Feature_Ax.clear()

    plt.figure().clear()
    plt.close(Snapshot_Fig)
    return None


# @profile
def bpt_figure(object_name, object_id, database_data, save_path):
    nii_over_halpha_ratio = database_data[7]
    oiii_over_hbeta_ratio = database_data[8]
    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]
    score = database_data[11]

    BPT_Plot = plt.figure(figsize=(16, 14), constrained_layout=False)
    AGNTestAX = plt.axes(box)

    plt.title(f"BPT Plot for {object_name}")
    plt.xlim(-2, 1)
    plt.ylim(-1, 1.5)

    # Kewley Theoretical Dividing Line
    x = np.linspace(-2, 0.25, 500)
    y = (0.61 / (x - 0.47)) + 1.19
    AGNTestAX.plot(x, y, color='k', label='Kewley 2001')
    AGNTestAX.annotate(text="AGN", xy=(0.25, -0.25), xycoords='data', horizontalalignment='center')
    AGNTestAX.annotate(text="Starforming", xy=(-1.5, -0.25), xycoords='data', horizontalalignment='center')

    AGNTestAX.set_xlabel(r"log([NII]/H$\alpha$)")
    AGNTestAX.set_ylabel(r"log([OIII]/H$\beta$)")

    AGNTestAX.axvline(x=(np.log10(0.6)), color='b', linestyle=':')
    AGNTestAX.axhline(y=(np.log10(3)), xmin=((np.log10(0.6) + 2) / 3), color='b', linestyle=':')

    AGNTestAX.annotate(text="Seyfert", xy=(0.25, 0.5), xycoords='data', horizontalalignment='center',
                       color='b')
    AGNTestAX.annotate(text="LINER", xy=(0.25, 0.4), xycoords='data', horizontalalignment='center',
                       color='b')

    # Kauffmann Empirical Dividing Line from SDSS Objects
    x = np.linspace(-2, 0.0, 500)
    y = (0.61 / (x - 0.05)) + 1.3
    AGNTestAX.plot(x, y, linestyle='--', color='k', label='Kauffmann 2003')

    AGNTestAX.scatter(nii_over_halpha_ratio, oiii_over_hbeta_ratio, marker='*', color='r', s=50)
    print("\n\n")
    print(nii_over_halpha_ratio, oiii_over_hbeta_ratio)
    print("\n\n")
    #sys.exit()

    #############################
    # Saving and wiping the plots
    #############################

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        BPT_Plot.savefig(f"{save_path}/BPT_Diagram_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
                         dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        BPT_Plot.savefig(f"{save_path}/BPT_Diagram_for_{object_name}_StrongFeVII.pdf",
                         dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        BPT_Plot.savefig(f"{save_path}/BPT_Diagram_for_{object_name}_Strong_FeXIV.pdf",
                         dpi=200, bbox_inches='tight', transparent = False)
    else:
        BPT_Plot.savefig(f"{save_path}/BPT_Diagram_for_{object_name}.pdf", dpi=200, bbox_inches='tight', transparent = False)

    plt.figure().clear()
    plt.close(BPT_Plot)

    return None


# @profile
def bpt_diagnostic_set_figure(object_name, object_id, database_data, save_path):
    nii_over_halpha_ratio = database_data[7]
    oiii_over_hbeta_ratio = database_data[8]

    sii_over_halpha_List = database_data[23]
    oi_over_halpha_List = database_data[24]

    print(object_name)
    print(nii_over_halpha_ratio)
    print(oiii_over_hbeta_ratio)
    print(sii_over_halpha_List)
    print(oi_over_halpha_List)

    fe_vii_flag = database_data[9]
    fe_xiv_flag = database_data[10]
    score = database_data[11]

    # BPT_Plot = plt.figure(figsize=(16, 14), constrained_layout=False)
    bpt_diagnostic_set_fig = plt.figure(figsize=(48, 14), constrained_layout=True)
    bpt_grid = bpt_diagnostic_set_fig.add_gridspec(1, 3, hspace=0.05, wspace=0.01)

    OIII_Hbeta_NII_Halpha_ax = bpt_diagnostic_set_fig.add_subplot(bpt_grid[0, 0])

    # plt.title(f"BPT Plot for {object_name}")
    plt.xlim(-2.25, 1)
    plt.ylim(-1.25, 1.5)

    # Kewley Theoretical Dividing Line
    x = np.linspace(-2.25, 0.25, 500)
    y = (0.61 / (x - 0.47)) + 1.19
    OIII_Hbeta_NII_Halpha_ax.plot(x, y, color='k', label='Kewley 2001')
    OIII_Hbeta_NII_Halpha_ax.annotate(text="AGN", xy=(0.25, -0.5), xycoords='data', horizontalalignment='center')
    OIII_Hbeta_NII_Halpha_ax.annotate(text="Starforming", xy=(-1.5, -0.5), xycoords='data',
                                      horizontalalignment='center')

    OIII_Hbeta_NII_Halpha_ax.set_xlabel(r"log([NII]/H$\alpha$)")
    OIII_Hbeta_NII_Halpha_ax.set_ylabel(r"log([OIII]/H$\beta$)")

    plt.axvline(x=(np.log10(0.6)), color='b', linestyle=':')
    plt.axhline(y=(np.log10(3)), xmin=((np.log10(0.6) + 2.25) / 3.25), color='b', linestyle=':')

    OIII_Hbeta_NII_Halpha_ax.annotate(text="Seyfert", xy=(0.25, 0.5), xycoords='data', horizontalalignment='center',
                                      color='b')
    OIII_Hbeta_NII_Halpha_ax.annotate(text="LINER", xy=(0.25, 0.4), xycoords='data', horizontalalignment='center',
                                      color='b')

    # Kauffmann Empirical Dividing Line from SDSS Objects
    x = np.linspace(-2.25, 0, 500)
    y = (0.61 / (x - 0.05)) + 1.3
    OIII_Hbeta_NII_Halpha_ax.plot(x, y, linestyle='--', color='k', label='Kauffmann 2003')

    plt.scatter(nii_over_halpha_ratio, oiii_over_hbeta_ratio, marker='*', color='r', s=750)


    OIII_Hbeta_SII_Halpha_ax = bpt_diagnostic_set_fig.add_subplot(bpt_grid[0, 1])
    plt.xlim(-2.25, 0.75)
    plt.ylim(-1.25, 1.5)
    OIII_Hbeta_SII_Halpha_ax.set_xlabel(r"log([SII]/H$\alpha$)")
    OIII_Hbeta_SII_Halpha_ax.set_ylabel(r"log([OIII]/H$\beta$)")
    # Main AGN line
    x = np.linspace(-2.25, 0.25, 500)
    y = 0.72 / (x - 0.32) + 1.30
    OIII_Hbeta_SII_Halpha_ax.plot(x, y, color='k')
    # LINER Seyfert 2 line
    x = np.linspace(-0.314594, 1.2, 500)
    y = (1.89 * x) + 0.76
    OIII_Hbeta_SII_Halpha_ax.plot(x, y, color='k')

    OIII_Hbeta_SII_Halpha_ax.annotate(text="HII", xy=(-0.75, -0.5), xycoords='data', horizontalalignment='center')
    OIII_Hbeta_SII_Halpha_ax.annotate(text="Seyfert", xy=(-0.75, 1.25), xycoords='data', horizontalalignment='center')
    OIII_Hbeta_SII_Halpha_ax.annotate(text="LINER", xy=(0.5, -0.5), xycoords='data', horizontalalignment='center')

    plt.scatter(sii_over_halpha_List, oiii_over_hbeta_ratio, marker='*', color='r', s=750)

    OIII_Hbeta_OI_Halpha_ax = bpt_diagnostic_set_fig.add_subplot(bpt_grid[0, 2])
    plt.xlim(-2.25, 1)
    plt.ylim(-1.25, 1.5)

    x = np.linspace(-2.25, -0.6, 500)
    # Main AGN line
    y = 0.73 / (x + 0.59) + 1.33
    OIII_Hbeta_OI_Halpha_ax.plot(x, y, color='k')

    # LINER Seyfert 2 line
    x = np.linspace(-1.12688, 0.5, 500)
    y = (1.18 * x) + 1.30
    OIII_Hbeta_OI_Halpha_ax.plot(x, y, color='k')

    OIII_Hbeta_OI_Halpha_ax.annotate(text="HII", xy=(-1.75, -0.5), xycoords='data', horizontalalignment='center')
    OIII_Hbeta_OI_Halpha_ax.annotate(text="Seyfert", xy=(-1.75, 1.25), xycoords='data', horizontalalignment='center')
    OIII_Hbeta_OI_Halpha_ax.annotate(text="LINER", xy=(0.5, -0.5), xycoords='data', horizontalalignment='center')

    OIII_Hbeta_OI_Halpha_ax.set_xlabel(r"log([OI]/H$\alpha$)")
    OIII_Hbeta_OI_Halpha_ax.set_ylabel(r"log([OIII]/H$\beta$)")

    plt.scatter(oi_over_halpha_List, oiii_over_hbeta_ratio, marker='*', color='r', s=750)

    #############################
    # Saving and wiping the plots
    #############################

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        bpt_diagnostic_set_fig.savefig(f"{save_path}/BPT_Set_Diagram_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
                                       dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        bpt_diagnostic_set_fig.savefig(f"{save_path}/BPT_Set_Diagram_for_{object_name}_StrongFeVII.pdf",
                                       dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        bpt_diagnostic_set_fig.savefig(f"{save_path}/BPT_Set_Diagram_for_{object_name}_Strong_FeXIV.pdf",
                                       dpi=200, bbox_inches='tight', transparent = False)
    else:
        bpt_diagnostic_set_fig.savefig(f"{save_path}/BPT_Set_Diagram_for_{object_name}.pdf",
                                       dpi=200, bbox_inches='tight', transparent = False)

    plt.figure().clear()
    plt.close(bpt_diagnostic_set_fig)

    return None


def line_diagnostic_figure(object_name, object_id, database_data, save_path):
    fe_vii_flag = database_data[8]
    fe_xiv_flag = database_data[9]

    lick_hdelta = database_data[12]
    lick_hdelta_err = database_data[13]

    lick_hgamma = database_data[14]
    lick_hgamma_err = database_data[15]

    line_diagnostic_plot = plt.figure(figsize=(16, 14), constrained_layout=False)
    lick_indices_ax = plt.axes(box)

    lick_indices_ax.set_xlabel(r'Lick H$\delta_A$')
    lick_indices_ax.set_ylabel(r'Lick H$\gamma_A$')

    lick_indices_ax.errorbar(lick_hdelta, lick_hgamma, xerr=lick_hdelta_err, yerr=lick_hgamma_err,
                             capthick=2, capsize=2, marker='o')

    lick_indices_ax.set_xlim(-6, 10)

    #############################
    # Saving and wiping the plots
    #############################

    if fe_vii_flag == 1 and fe_xiv_flag == 1:
        line_diagnostic_plot.savefig(f"{save_path}/Line_Diagnostic_Plot_for_{object_name}_StrongFeVII_StrongFeXIV.pdf",
                                     dpi=200, bbox_inches='tight', transparent = False)

    elif fe_vii_flag == 1:
        line_diagnostic_plot.savefig(f"{save_path}/Line_Diagnostic_Plot_for_{object_name}_StrongFeVII.pdf",
                                     dpi=200, bbox_inches='tight', transparent = False)

    elif fe_xiv_flag == 1:
        line_diagnostic_plot.savefig(f"{save_path}/Line_Diagnostic_Plot_for_{object_name}_Strong_FeXIV.pdf",
                                     dpi=200, bbox_inches='tight', transparent = False)
    else:
        line_diagnostic_plot.savefig(f"{save_path}/Line_Diagnostic_Plot_for_{object_name}.pdf", dpi=200,
                                     bbox_inches='tight', transparent = False)

    plt.figure().clear()
    plt.close(line_diagnostic_plot)

    return None


########################
# Line 'list' Dictionaries
########################

Spectral_Lines_For_Scoring_Air = Hirogen_Functions.lines_for_scoring()
Spectral_Lines_For_Analysis_Air = Hirogen_Functions.lines_for_analysis()

Spectral_Lines_For_Analysis_Air_Names = list(Spectral_Lines_For_Analysis_Air.keys())

Spectral_Lines_For_Snapshot_Plot_1 = Hirogen_Functions.primary_lines()
Spectral_Lines_For_Snapshot_Plot_2 = Hirogen_Functions.secondary_lines()
Spectral_Lines_For_Snapshot_Plot_3 = Hirogen_Functions.tertiary_lines()

Line_Labels_1 = Hirogen_Functions.primary_line_labels()
Line_Labels_2 = Hirogen_Functions.secondary_line_labels()
Line_Labels_3 = Hirogen_Functions.tertiary_line_labels()

Comparison_Lines = Hirogen_Functions.comparison_lines()
Comparison_Line_Labels = Hirogen_Functions.comparison_plot_line_labels_short()

Zoom_in_Presentation_Lines = Hirogen_Functions.presentation_zoom_lines()
Zoom_in_Presentation_Line_Labels = Hirogen_Functions.presentation_zoom_line_labels()

print("Function Configuration Complete")

In [ ]:
##############
# Data Readin
#############

########################
# Main Savepaths
########################

Timestamp = str(datetime.today().date())

MainSavePath = os.path.join(f"Hirogen_Outputs/{Timestamp}")

Hirogen_Functions.directory_creation(MainSavePath)

PerTableSavePath = os.path.join(MainSavePath, f"{TableID}")
Hirogen_Functions.directory_creation(PerTableSavePath)

if not NERSC_Flag and not Local_DESI:

    Data = Hirogen_Functions.database_connection(user=Database_User, password=Database_Password, database=Database)
    cursor = Data.cursor()

    # The 'errors' here are because PyCharm can't seem to cope with the Table name being variable rather than a
    # straight up sting - it will run properly though. To test is the schema is right swap out the variable for the
    # table name temporarily

    Candidate_Threshold = 0  # Local override

    IDs = (1021306429161629696)
    # QUERY
    cursor.execute(
        "SELECT SDSS_ShortName, DR16_Spectroscopic_ID, spec_Plate, spec_MJD, spec_FiberID, "
        "z_SDSS_spec, generalised_extinction, lin_con_NII_over_Halpha, lin_con_OIII_over_Hbeta, "
        "Strong_FeVII_Flag, Strong_FeXIV_Flag, ECLE_Candidate_Score, Lick_HDelta_Index, Lick_HDelta_Index_Err, "
        "Lick_HGamma_Index, Lick_HGamma_Index_Err, survey, run2d, Standard_Inclusion, Path_Override_Flag, "
        "Path_Override, Follow_Up_ID, Smoothing_Override, lin_con_SII_over_Halpha, lin_con_OI6300_over_Halpha, "
        "lin_con_OIII5007_over_OIII4959, z_corrected_flag, extinction_corrected_flag, Settings_Config "
        f"FROM {Database}.{TableID} "
        #"WHERE Manually_Inspected_Flag != -10 AND ECLE_Candidate_Score >= 7 "
        # "WHERE lin_con_pEQW_Halpha = 0"
        #"WHERE (ECLE_Candidate_Score >= %s AND Standard_Inclusion = 1) % Candidate_Threshold"
        # "OR (Strong_FeVII_Flag = 1 AND Standard_Inclusion = 1) "
        # "OR (Strong_FeXIV_Flag = 1 AND Standard_Inclusion = 1)"
        #f"WHERE DR16_Spectroscopic_ID IN {IDs}"
        #f"WHERE DR16_Spectroscopic_ID = {IDs}"
        #"WHERE (Standard_Inclusion = 1)"
        #"WHERE (Follow_Up_ID = 0)"
        #"WHERE (Follow_Up_ID = 1)"
        #"WHERE Nickname = 'Charizard'"
        #f"WHERE DR16_Spectroscopic_ID != '8598733000363233280' and ECLE_Candidate_Score >= 7"
        # This apparently leaves me open to SQL injection, though I have yet to find out how to do this properly
        # successfully and its Friday afternoon and my head hurts so I legit do not care
    )

    Candidate_Data = cursor.fetchall()

    print(
        f"Based on the scoring threshold set, {len(Candidate_Data)} objects pass the initial cut and will have "
        f"plots produced.")

    if len(Candidate_Data) >= 1:

        Object_Name_List = [item[0] for item in Candidate_Data]
        Object_Spec_ID_List = [item[1] for item in Candidate_Data]
        Plate_List = [item[2] for item in Candidate_Data]
        MJD_List = [item[3] for item in Candidate_Data]
        FiberID_List = [item[4] for item in Candidate_Data]
        Redshift_List = [item[5] for item in Candidate_Data]
        Extinction_List = [item[6] for item in Candidate_Data]
        NII_over_Halpha_List = [item[7] for item in Candidate_Data]
        OIII_over_Hbeta_List = [item[8] for item in Candidate_Data]
        Strong_FeVII_Flag_List = [item[9] for item in Candidate_Data]
        Strong_FeXIV_Flag_List = [item[10] for item in Candidate_Data]
        ECLE_Candidate_Score_List = [item[11] for item in Candidate_Data]
        Lick_Hdelta = [item[12] for item in Candidate_Data]
        Lick_Hdelta_Err = [item[13] for item in Candidate_Data]
        Lick_Hgamma = [item[14] for item in Candidate_Data]
        Lick_Hgamma_Err = [item[15] for item in Candidate_Data]
        Survey_List = [item[16] for item in Candidate_Data]
        run2d_List = [item[17] for item in Candidate_Data]
        Standard_Inclusion_List = [item[18] for item in Candidate_Data]
        Path_Override_Flag_List = [item[19] for item in Candidate_Data]
        Path_Override_List = [item[20] for item in Candidate_Data]
        Follow_Up_ID_List = [item[21] for item in Candidate_Data]
        Smoothing_Override_List = [item[22] for item in Candidate_Data]
        SII_over_Halpha_List = [item[23] for item in Candidate_Data]
        OI_over_Halpha_List = [item[24] for item in Candidate_Data]
        OIII5007_over_OIII4959_List = [item[25] for item in Candidate_Data]
        z_Correction_Override_List = [item[26] for item in Candidate_Data]
        Extinction_Correction_Override_List = [item[27] for item in Candidate_Data]
        Stored_Config_Settings_List = [item[28] for item in Candidate_Data]

    else:
        print("Candidate_Data Length error: Check and try again.")
        sys.exit()

    if not Candidate_Data:
        print(f"No Candidate Data Found.\nThere are no candidates within the Table: {TableID}\nExiting")
        sys.exit()

    if Follow_Up:
        FilePaths = [f"./{x}.ascii" for x in Object_Name_List]
    else:
        FilePaths = Hirogen_Functions.sdss_spectra_file_path_generator(
            Main_Spectra_Path, Plate_List, MJD_List, FiberID_List, Survey_List, run2d_List,
            override_path_flag_list=Path_Override_Flag_List, override_path_list=Path_Override_List
        )
    

    Number_of_Objects = len(FilePaths)

    print(FilePaths)

elif NERSC_Flag:
    # When running on NERSC mode the script will look for a file in the format: 'Storage_File_for_BLOCK_ID_{BLOCK_ID}_BGS.ascii' to pull the information from
    # It will read in this data and then construct the necessary file paths to the actual spectra

    Storage_File = f"Storage_File_for_BLOCK_ID_{BLOCK_ID}_BGS.ascii"

    Data_Input = Hirogen_Functions.desi_datafile_reader(Storage_File)

    #####
    # Data Retrieval
    #####
    DESI_TARGET_ID = Data_Input[0]
    DATE = Data_Input[1]
    TILE = Data_Input[2]
    PETAL = Data_Input[3]
    FIBERID = Data_Input[4]
    FIBERSTATUS = Data_Input[5]
    
    PER_FILE_INDEX = Data_Input[6]

    RA = Data_Input[7]
    DEC = Data_Input[8]

    REDSHIFT = Data_Input[9]
    REDSHIFT_ERR = Data_Input[10]

    EBV = Data_Input[11]

    # Line pEQWs

    Halpha_pEQW = Data_Input[12]
    Hbeta_pEQW = Data_Input[13]
    Hgamma_pEQW = Data_Input[14]
    Hdelta_pEQW = Data_Input[15]

    NII_6548_pEQW = Data_Input[16]
    NII_6584_pEQW = Data_Input[17]

    FeVII_3759_pEQW = Data_Input[18]
    FeVII_5160_pEQW = Data_Input[19]
    FeVII_5722_pEQW = Data_Input[20]
    FeVII_6088_pEQW = Data_Input[21]
    FeX_6346_pEQW = Data_Input[22]
    FeXI_7894_pEQW = Data_Input[23]
    FeXIV_5304_pEQW = Data_Input[24]

    OI_6300_pEQW = Data_Input[25]
    OII_3728_pEQW = Data_Input[26]
    OIII_4959_pEQW = Data_Input[27]
    OIII_5007_pEQW = Data_Input[28]

    HeI_4478_pEQW = Data_Input[29]
    HeII_4686_pEQW = Data_Input[30]

    NaID_pEQW = Data_Input[31]

    SII_6717_pEQW = Data_Input[32]
    SII_6731_pEQW = Data_Input[33]
    SII_6717_6731_pEQW = Data_Input[34]

    # Line Fluxes

    Halpha_LineFlux = Data_Input[35]
    Hbeta_LineFlux = Data_Input[36]
    Hgamma_LineFlux = Data_Input[36]
    Hdelta_LineFlux = Data_Input[38]

    NII_6548_LineFlux = Data_Input[39]
    NII_6584_LineFlux = Data_Input[40]

    FeVII_3759_LineFlux = Data_Input[41]
    FeVII_5160_LineFlux = Data_Input[42]
    FeVII_5722_LineFlux = Data_Input[43]
    FeVII_6088_LineFlux = Data_Input[44]
    FeX_6346_LineFlux = Data_Input[45]
    FeXI_7894_LineFlux = Data_Input[46]
    FeXIV_5304_LineFlux = Data_Input[47]

    OI_6300_LineFlux = Data_Input[48]
    OII_3728_LineFlux = Data_Input[49]
    OIII_4959_LineFlux = Data_Input[50]
    OIII_5007_LineFlux = Data_Input[51]

    HeI_4478_LineFlux = Data_Input[52]
    HeII_4686_LineFlux = Data_Input[53]

    NaID_LineFlux = Data_Input[54]

    SII_6717_LineFlux = Data_Input[55]
    SII_6731_LineFlux = Data_Input[56]
    SII_6717_6731_LineFlux = Data_Input[57]

    # Ratios
    NII_over_Halpha_Ratio = Data_Input[58]
    OIII_over_Hbeta_Ratio = Data_Input[59]
    SII_over_Halpha_Ratio = Data_Input[60]
    OI_over_Halpha_Ratio = Data_Input[61]
    OIII_over_OIII_Ratio = Data_Input[62]
    He_over_Halpha_Ratio = Data_Input[63]

    # Lick Indices
    D4000 = Data_Input[64]
    D4000_Err = Data_Input[65]
    Lick_HGAMMA = Data_Input[66]
    Lick_HGAMMA_Err = Data_Input[67]
    Lick_HDELTA = Data_Input[68]
    Lick_HDELTA_Err = Data_Input[69]

    # Strong Line Flags
    Strong_FeVII_Flag = Data_Input[70]
    Strong_FeX_Flag = Data_Input[71]
    Strong_FeXI_Flag = Data_Input[72]
    Strong_FeXIV_Flag = Data_Input[73]

    # Pixel Information
    All_Flagged_Pixel_Percentage = Data_Input[74]
    Counted_Flagged_Pixel_Percentage = Data_Input[75]

    # Object Classification Flags
    Quiescent_Flag = Data_Input[76]
    Balmer_Strong_Flag = Data_Input[77]
    Balmer_Moderately_Strong_Flag = Data_Input[78]

    # ECLE Score

    ECLE_Score = Data_Input[79]

    # Flux Information
    # G
    G_FiberFlux = Data_Input[80]
    G_Flux_Ivar = Data_Input[81]

    # R
    R_FiberFlux = Data_Input[82]
    R_Flux_Ivar = Data_Input[83]

    # Z
    Z_FiberFlux = Data_Input[84]
    Z_Flux_Ivar = Data_Input[85]

    # W1
    W1_Flux = Data_Input[86]
    W1_Flux_Ivar = Data_Input[87]

    # W2
    W2_Flux = Data_Input[88]
    W2_Flux_Ivar = Data_Input[89]
    
    # Spectrum Identification ID - Formatted: '{DESI_TARGET_ID[ii]}_{TILE[ii]}_{PETAL[ii]}_{FIBERID[ii]}' : Allows for objects to be treated independently if they are observed in differnet locations e.g. on 2 different tiles
    Identification_IDs = Data_Input[90]
    
    # Stored Config Settings
    Stored_Config_Settings = Data_Input[91]
    
    Origin_Files = []

    Number_of_Objects = 0
    
    Fiber_IDs = []
    Spectra_To_Plot_List_Store = []
    
    # This is to preserve the same order as the plots expect for SDSS data - Several of these will not be used for DESI
    Object_Name_List = []
    Object_Spec_ID_List = []
    Plate_List = []
    MJD_List = []
    FiberID_List = []
    Redshift_List = []
    Extinction_List = []
    NII_over_Halpha_List = []
    OIII_over_Hbeta_List = []
    Strong_FeVII_Flag_List = []
    Strong_FeXIV_Flag_List = []
    ECLE_Candidate_Score_List = []
    Lick_Hdelta = []
    Lick_Hdelta_Err = []
    Lick_Hgamma = []
    Lick_Hgamma_Err = []
    Survey_List = []
    run2d_List = []
    Standard_Inclusion_List = []
    Path_Override_Flag_List = []
    Path_Override_List = []
    Follow_Up_ID_List = []
    Smoothing_Override_List = []
    SII_over_Halpha_List = []
    OI_over_Halpha_List = []
    OIII5007_over_OIII4959_List = []
    Stored_Config_Settings_List = []
    Ident_IDs_List = []
    
    Candidate_Threshold = 7 # Local Override
    
    if Start_Tile == None:
        Start_Tile = np.min(TILE)
    if End_Tile == None:
        End_Tile = np.max(TILE)
    
    for ii, item in enumerate(DESI_TARGET_ID):
        
        if ECLE_Score[ii] >= Candidate_Threshold:
            Origin_Files.append(f"{TILE[ii]}/{DATE[ii]}/coadd-{PETAL[ii]}-{TILE[ii]}-thru{DATE[ii]}.fits")
            Number_of_Objects += 1
            
            # General info for each candidate
            Object_Name_List.append(DESI_TARGET_ID[ii])
            Object_Spec_ID_List.append(DESI_TARGET_ID[ii]) #'Name' and ID are the same for DESI Objects
            Plate_List.append(TILE[ii]) # Plate == Tile
            MJD_List.append(DATE[ii]) # MJD given as a date for now - will likely expand this to have MJDs and Dates for both SDSS and DESI
            FiberID_List.append(FIBERID[ii])
            Redshift_List.append(REDSHIFT[ii])
            Extinction_List.append(EBV[ii])
            NII_over_Halpha_List.append(NII_over_Halpha_Ratio[ii])
            OIII_over_Hbeta_List.append(OIII_over_Hbeta_Ratio[ii])
            Strong_FeVII_Flag_List.append(Strong_FeVII_Flag[ii])
            Strong_FeXIV_Flag_List.append(Strong_FeXIV_Flag[ii])
            ECLE_Candidate_Score_List.append(ECLE_Score[ii])
            Lick_Hdelta.append(Lick_HDELTA[ii])
            Lick_Hdelta_Err.append(Lick_HDELTA_Err[ii])
            Lick_Hgamma.append(Lick_HGAMMA[ii])
            Lick_Hgamma_Err.append(Lick_HDELTA_Err[ii])
            Survey_List.append('DESI SV')
            run2d_List.append('-999')
            Standard_Inclusion_List.append(0)
            Path_Override_Flag_List.append(0)
            Path_Override_List.append(0)
            Follow_Up_ID_List.append(0)
            Smoothing_Override_List.append(0)
            SII_over_Halpha_List.append(SII_over_Halpha_Ratio[ii])
            OI_over_Halpha_List.append(OI_over_Halpha_Ratio[ii])
            OIII5007_over_OIII4959_List.append(OIII_over_OIII_Ratio[ii])
            Stored_Config_Settings_List.append(Stored_Config_Settings[ii])
            Ident_IDs_List.append(Identification_IDs[ii])

    z_Correction_Override_List = [np.float64(0)] * len(Object_Name_List)
    Extinction_Correction_Override_List = [np.float64(0)] * len(Object_Name_List)

    Included = set()
    Needed_Coadd_Files = [x for x in Origin_Files if x not in Included and (Included.add(x) or True)]
    File_Counts = Counter(Origin_Files)
    
    for ii, item in enumerate(Needed_Coadd_Files):
        
        Fiber_ID_List = []
        
        for jj, spectrum in enumerate(DESI_TARGET_ID):
            
            if ECLE_Score[jj] >= Candidate_Threshold and Needed_Coadd_Files[ii] == f"{TILE[jj]}/{DATE[jj]}/coadd-{PETAL[jj]}-{TILE[jj]}-thru{DATE[jj]}.fits":
                Fiber_ID_List.append(FIBERID[jj])
                
        Spectra_To_Plot_List_Store.append(Fiber_ID_List)

    allwave = None
    allflux = None
    allivar = None
    allfmap = None

    for ii, item in enumerate(Needed_Coadd_Files):

        coadd_filepath = f"{redux}/{Needed_Coadd_Files[ii]}"
        pspectra = read_spectra(coadd_filepath)
        cspectra = coadd_cameras(pspectra)
        fibermap = cspectra.fibermap

        isECLE_Candidate = np.isin(fibermap['FIBER'], Spectra_To_Plot_List_Store[ii])

        if allwave is None:

            allwave = cspectra.wave['brz']
            allflux = cspectra.flux['brz'][isECLE_Candidate]
            allivar = cspectra.ivar['brz'][isECLE_Candidate]
            #allfmap = fibermap[isECLE_Candidate]

        else:

            allflux = np.vstack([allflux, cspectra.flux['brz'][isECLE_Candidate]])
            allivar = np.vstack([allivar, cspectra.ivar['brz'][isECLE_Candidate]])
            #allmask = np.vstack([allmask, cspectra.mask['brz'][isECLE_Candidate]])
            #allfmap = np.vstack([allfmap, fibermap[isECLE_Candidate]])
    
    Candidate_Data = []
    for ii, item in enumerate(Object_Name_List):
        Data_Store = [Object_Name_List[ii],Object_Spec_ID_List[ii],Plate_List[ii],MJD_List[ii],FiberID_List[ii],Redshift_List[ii],Extinction_List[ii],NII_over_Halpha_List[ii],OIII_over_Hbeta_List[ii],Strong_FeVII_Flag_List[ii],Strong_FeXIV_Flag_List[ii],ECLE_Candidate_Score_List[ii],Lick_Hdelta[ii],Lick_Hdelta_Err[ii],Lick_Hgamma[ii],Lick_Hgamma_Err[ii],Survey_List[ii],run2d_List[ii],Standard_Inclusion_List[ii],Path_Override_Flag_List[ii],Path_Override_List[ii],Follow_Up_ID_List[ii],Smoothing_Override_List[ii],SII_over_Halpha_List[ii],OI_over_Halpha_List[ii], Stored_Config_Settings_List[ii], Ident_IDs_List[ii]]
        Candidate_Data.append(Data_Store)
        
    if Number_of_Objects == 0:
        print("No identified candidates in this file")

else: # Local DESI Files - Pulls data from a database table

    Standard_Path_Flag = False

    Data = Hirogen_Functions.database_connection(user=Database_User, password=Database_Password, database=Database)
    cursor = Data.cursor()

    # The red 'errors' here are because PyCharm can't seem to cope with the Table name being variable rather than a
    # straight up sting - it will run properly though. To test is the schema is right swap out the variable for the
    # table name temporarily

    Candidate_Threshold = 7  # Local override

    IDs = (1955789619387721728, 5)

    # QUERY
    cursor.execute(
        "SELECT Nickname, DESI_Target_ID, TILE, Cumulative_Date, FIBER_ID, "
        "z_best, EBV, lin_con_NII_over_Halpha, lin_con_OIII_over_Hbeta, "
        "Strong_FeVII_Flag, Strong_FeXIV_Flag, ECLE_Candidate_Score, Lick_HDelta_Index, Lick_HDelta_Index_Err, "
        "Lick_HGamma_Index, Lick_HGamma_Index_Err, Standard_Inclusion, Path_Override_Flag, "
        "Path_Override, Smoothing_Override, lin_con_SII_over_Halpha, lin_con_OI6300_over_Halpha, "
        "lin_con_OIII5007_over_OIII4959, Settings_Config, IDENT_ID, Follow_Up_ID, z_corrected_flag ,extinction_corrected_flag "
        f"FROM {Database}.{TableID} "
        #"WHERE Manually_Inspected_Flag != -10 AND ECLE_Candidate_Score >= 7 "
        # "WHERE lin_con_pEQW_Halpha = 0"
        #"WHERE (ECLE_Candidate_Score >= %s AND Standard_Inclusion = 1) % Candidate_Threshold"
        # "OR (Strong_FeVII_Flag = 1 AND Standard_Inclusion = 1) "
        # "OR (Strong_FeXIV_Flag = 1 AND Standard_Inclusion = 1)"
        # f"WHERE DR16_Spectroscopic_ID IN {IDs}"
        #"WHERE (Standard_Inclusion = 1)"
        #"WHERE (Follow_Up_ID = 0)"
        #"WHERE Nickname = 'Charizard'"
        # This apparently leaves me open to SQL injection, though I have yet to find out how to do this properly
        # successfully and its Friday afternoon and my head hurts so I legit do not care
    )

    Candidate_Data = cursor.fetchall()

    print(
        f"Based on the scoring threshold set, {len(Candidate_Data)} objects pass the initial cut and will have "
        f"plots produced.")

    if len(Candidate_Data) >= 1:

        Object_Name_List = [item[0] for item in Candidate_Data]
        Object_Spec_ID_List = [item[1] for item in Candidate_Data]
        Plate_List = [item[2] for item in Candidate_Data]
        MJD_List = [item[3] for item in Candidate_Data]
        FiberID_List = [item[4] for item in Candidate_Data]
        Redshift_List = [item[5] for item in Candidate_Data]
        Extinction_List = [item[6] for item in Candidate_Data]
        NII_over_Halpha_List = [item[7] for item in Candidate_Data]
        OIII_over_Hbeta_List = [item[8] for item in Candidate_Data]
        Strong_FeVII_Flag_List = [item[9] for item in Candidate_Data]
        Strong_FeXIV_Flag_List = [item[10] for item in Candidate_Data]
        ECLE_Candidate_Score_List = [item[11] for item in Candidate_Data]
        Lick_Hdelta = [item[12] for item in Candidate_Data]
        Lick_Hdelta_Err = [item[13] for item in Candidate_Data]
        Lick_Hgamma = [item[14] for item in Candidate_Data]
        Lick_Hgamma_Err = [item[15] for item in Candidate_Data]

        Standard_Inclusion_List = [item[16] for item in Candidate_Data]
        Path_Override_Flag_List = [item[17] for item in Candidate_Data]
        Path_Override_List = [item[18] for item in Candidate_Data]

        Smoothing_Override_List = [item[19] for item in Candidate_Data]
        SII_over_Halpha_List = [item[20] for item in Candidate_Data]
        OI_over_Halpha_List = [item[21] for item in Candidate_Data]
        OIII5007_over_OIII4959_List = [item[22] for item in Candidate_Data]

        Stored_Config_Settings_List = [item[23] for item in Candidate_Data]
        Ident_IDs_List = [item[24] for item in Candidate_Data]

        Follow_Up_ID_List = [item[25] for item in Candidate_Data]
        z_Correction_Override_List = [item[26] for item in Candidate_Data]
        Extinction_Correction_Override_List = [item[27] for item in Candidate_Data]

        Survey_List = []
        run2d_List = []

        for jj, object in enumerate(Object_Name_List):
            Survey_List.append('DESI SV')
            run2d_List.append('-999')

        Candidate_Data = []
        for ii, item in enumerate(Object_Name_List):
            Data_Store = [Object_Name_List[ii],Object_Spec_ID_List[ii],Plate_List[ii],MJD_List[ii],FiberID_List[ii],Redshift_List[ii],Extinction_List[ii],NII_over_Halpha_List[ii],OIII_over_Hbeta_List[ii],Strong_FeVII_Flag_List[ii],Strong_FeXIV_Flag_List[ii],ECLE_Candidate_Score_List[ii],Lick_Hdelta[ii],Lick_Hdelta_Err[ii],Lick_Hgamma[ii],Lick_Hgamma_Err[ii],Survey_List[ii],run2d_List[ii],Standard_Inclusion_List[ii],Path_Override_Flag_List[ii],Path_Override_List[ii],Follow_Up_ID_List[ii],Smoothing_Override_List[ii],SII_over_Halpha_List[ii],OI_over_Halpha_List[ii], Stored_Config_Settings_List[ii], Ident_IDs_List[ii]]
            Candidate_Data.append(Data_Store)

    else:
        print("Candidate_Data Length error: Check and try again.")
        sys.exit()

    if not Candidate_Data:
        print(f"No Candidate Data Found.\nThere are no candidates within the Table: {TableID}\nExiting")
        sys.exit()

    FilePaths = Hirogen_Functions.sdss_spectra_file_path_generator(
        Main_Spectra_Path, Plate_List, MJD_List, FiberID_List, Survey_List, run2d_List,
        override_path_flag_list=Path_Override_Flag_List, override_path_list=Path_Override_List
    )

    Number_of_Objects = len(FilePaths)

    print(FilePaths)

In [ ]:
for Spectrum_Number in tqdm(range(Number_of_Objects), desc="Number of spectra processed"):

    # This looks to be broken for some reason
    """
    if Stored_Config_Settings[Spectrum_Number] != Settings_Config:
        print(f'Config mismatch: Stored {Stored_Config_Settings_List[Spectrum_Number]}\t Current {Settings_Config}\n')
        sys.exit()
    """

    Candidate_Score = 0
    Recorded_Database_Score = ECLE_Candidate_Score_List[Spectrum_Number]

    if TableID in ["SDSS_Confirmed_Objects", "SDSS_Confirmed_Objects_Rebin_Test", "SDSS_scaled_objects", "DESI_LOCAL"]:
        ObjectName = Object_Name_List[Spectrum_Number]
    else:
        ObjectName = Object_Spec_ID_List[Spectrum_Number]

    Object_Spec_Id = Object_Spec_ID_List[Spectrum_Number]

    """
    if Object_Spec_Id != 39627787308371326:
        Spectrum_Number += 1
        continue
    """

    if Smoothing_Override_List[Spectrum_Number] == 1:
        Smoothing = False
    else:
        Smoothing = Default_Smoothing

    #########################
    # Per Object Savepaths
    #########################
    if NERSC_Flag:
        PerObjectSavePath = os.path.join(PerTableSavePath, f"{Ident_IDs_List[Spectrum_Number]}")
    else:
        PerObjectSavePath=os.path.join(PerTableSavePath, f"{ObjectName}")
    Hirogen_Functions.directory_creation(PerTableSavePath)

    PlotsSavePath = os.path.join(PerObjectSavePath, "Plots")
    Hirogen_Functions.directory_creation(PlotsSavePath)

    #######################
    # Creation of Line report if desired
    #######################

    if Line_Report:
        line_scoring_report = open(f'{PerObjectSavePath}/Line_Scoring_Report_For_{ObjectName}.csv', 'w')
        line_scoring_report.write(
            f'Line\tpEQW\tPeak Mean\tPeak Max\tPeak Max Location\tRegion Mean\tRegion Max\tRegion Max Location\t'
            f'Condition\n')

    #######################
    # Data Read
    #######################

    if not NERSC_Flag:

        if Local_DESI:
            Spectral_Data = Hirogen_Functions.ascii_spectrum_reader(
                filepath=FilePaths[Spectrum_Number],
                z=Redshift_List[Spectrum_Number],
                extinction=Extinction_List[Spectrum_Number],
                smoothing=Smoothing,
                z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
            )

        elif Standard_Inclusion_List[Spectrum_Number] == 0:
            FilePaths[Spectrum_Number] = Path_Override_List[Spectrum_Number]

            # I'm going to assume that non standard files are .txt or .ascii with three columns
            # wave, flux, flux err, unless specifically told otherwise via the 'survey' column in the database
            # wavelength should be given in angs

            if Survey_List[Spectrum_Number] in ('MMT_Fits', 'Fits', 'FITS'):

                    Spectral_Data = Hirogen_Functions.fits_spectrum_reader(
                    filepath=FilePaths[Spectrum_Number],
                    z=Redshift_List[Spectrum_Number],
                    extinction=Extinction_List[Spectrum_Number],
                    smoothing=Smoothing,
                    z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                    extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                )

            else:

                Spectral_Data = Hirogen_Functions.ascii_spectrum_reader(
                    filepath=FilePaths[Spectrum_Number],
                    z=Redshift_List[Spectrum_Number],
                    extinction=Extinction_List[Spectrum_Number],
                    smoothing=Smoothing,
                    z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                    extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                )

        else:
            if Follow_Up:
                    Spectral_Data = Hirogen_Functions.ascii_spectrum_reader(
                        filepath=FilePaths[Spectrum_Number],
                        z=Redshift_List[Spectrum_Number],
                        extinction=Extinction_List[Spectrum_Number],
                        smoothing=Smoothing,
                        z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                        extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                    )
            else:
                Spectral_Data = Hirogen_Functions.sdss_spectrum_reader(
                    filepath=FilePaths[Spectrum_Number],
                    z=Redshift_List[Spectrum_Number],
                    extinction=Extinction_List[Spectrum_Number],
                    smoothing=Smoothing,
                    z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                    extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                )

    else:


        Spectral_Data = [
            Hirogen_Functions.rest_wavelength_converter(allwave, Redshift_List[Spectrum_Number])* u.AA,
            (allflux[Spectrum_Number]* u.Unit('erg cm-2 s-1 AA-1') / extinction_model.extinguish(allwave* u.AA, Ebv=EBV[Spectrum_Number])),
            allivar[Spectrum_Number]
        ]

    Rest_Wavelength = Spectral_Data[0]
    Flux = Spectral_Data[1]
    Error = Spectral_Data[2]

    if Local_DESI: # Makes sure the DESI spectra are scaled to the same units as the SDSS spectra
        Flux = Flux / 1e17
        Error = Flux / 1e17

    #Specific denoising steps

    ###########
    # Rebinning
    ###########

    if Spectres_Rebin:
        spectres_data = Hirogen_Functions.spectres_rebin(flux=Flux.value, wave=Rest_Wavelength.value, desired_res=Rebin_Res)
        Flux = spectres_data[0] * u.Unit('erg cm-2 s-1 AA-1')
        Rest_Wavelength = spectres_data[1] * u.Unit('AA')

    ###########
    # FFT Denoise
    ###########

    if FFT_Denoise:
        filtered_flux = Hirogen_Functions.fft_filter_signal(Flux.value, threshold = FFT_Threshold)
        Flux = filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

    ###########
    # Gaussian Smoothing
    ###########

    if Gaussian_Smooth:
        gaussian_smoothed_flux = Hirogen_Functions.gaussian_smoothing(Flux.value, sigma_value = Gaussian_Sigma)
        Flux = gaussian_smoothed_flux * u.Unit('erg cm-2 s-1 AA-1')

    ###########
    # Median Filtering
    ###########

    if Median_Filter: # Runs a median filter, defaults to a kernel of size 3 but can be set via the function call
            filtered_flux = Hirogen_Functions.spectral_median_filter(Flux, med_filter_kernel = Median_Filter_Kernel)
            Flux = filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

    ######
    # General check and removal of -ve flux points
    ######
    """
    for ii, item in enumerate(Flux):
        if Flux[ii].value < 0:
            Flux[ii] = np.nan * u.Unit('erg cm-2 s-1 AA-1')
    """


    for ii, item in enumerate(Spectral_Lines_For_Analysis_Air):

        Line_Location = Spectral_Lines_For_Analysis_Air[item][0]

        # Velocity shift from line
        Shift = (((np.array(Rest_Wavelength) * c) / Line_Location) - c)

        # Cut down the region for analysis
        # This may need to be tweaked currently 75 angstroms either side of the line
        # Will default to nans if spectrum does not extend to fully include these regions

        if Rest_Wavelength[-1].value < Line_Location - 75 or Rest_Wavelength[1].value > Line_Location + 75:

            if debug:
                print(f"Analysis region \n{Spectral_Lines_For_Analysis_Air_Names[ii]} "
                      f"is not fully covered in the rest frame.")

            Spectral_Lines_For_Analysis_Air[item][2] = np.nan
            Spectral_Lines_For_Analysis_Air[item][3] = np.nan
            Spectral_Lines_For_Analysis_Air[item][4] = np.nan
            Spectral_Lines_For_Analysis_Air[item][5] = np.nan
            Spectral_Lines_For_Analysis_Air[item][6] = np.nan
            Spectral_Lines_For_Analysis_Air[item][7] = np.nan
            Spectral_Lines_For_Analysis_Air[item][8] = -999
            Spectral_Lines_For_Analysis_Air[item][9] = np.nan
            Spectral_Lines_For_Analysis_Air[item][10] = np.nan
            Spectral_Lines_For_Analysis_Air[item][11] = np.nan
            Spectral_Lines_For_Analysis_Air[item][12] = np.nan
            Spectral_Lines_For_Analysis_Air[item][13] = np.nan
            Spectral_Lines_For_Analysis_Air[item][14] = np.nan
            Spectral_Lines_For_Analysis_Air[item][15] = np.nan
            Spectral_Lines_For_Analysis_Air[item][16] = np.nan

        else:
            Shift_Region, Wave_Region, Flux_Region, Error_Region = Hirogen_Functions.region_cutter(
                shift=Shift, wave=Rest_Wavelength,
                flux=Flux,
                low_cut=Line_Location - Lower_Wave,
                high_cut=Line_Location + Upper_Wave,
                mode='Wavelength'
            )

            # Generate and remove a continuum fit from the flux data - currently linear other functions seem worse
            Continuum = Hirogen_Functions.continua_maker(
                spec_region=Wave_Region,
                flux=Flux_Region,
                # shift=Shift_Region,
                line_name=list(Spectral_Lines_For_Analysis_Air)[ii],
                line_loc=Line_Location,
                object_name=ObjectName
            )

            Spectral_Lines_For_Analysis_Air[item][12] = [Continuum[2][0],
                                                         (((Continuum[2][0] * c) / Line_Location) - c)]
            Spectral_Lines_For_Analysis_Air[item][13] = [Continuum[2][1],
                                                         (((Continuum[2][1] * c) / Line_Location) - c)]
            Spectral_Lines_For_Analysis_Air[item][14] = [Continuum[2][2],
                                                         (((Continuum[2][2] * c) / Line_Location) - c)]
            Spectral_Lines_For_Analysis_Air[item][15] = [Continuum[2][3],
                                                         (((Continuum[2][3] * c) / Line_Location) - c)]

            # Measure the resulting line flux and pEQW of the feature

            pEQW_LinCon_Spec, Line_Flux = Hirogen_Functions.eqw_measurement(
                flux = Flux_Region,
                true_continuum = Continuum[0],
                scaled_flux=Continuum[1],
                xaxis=Wave_Region,
                line_loc=Line_Location
            )

            print(pEQW_LinCon_Spec, Line_Flux)

            # Store the results in the main dictionary

            Spectral_Lines_For_Analysis_Air[item][2] = Shift
            Spectral_Lines_For_Analysis_Air[item][3] = Shift_Region
            Spectral_Lines_For_Analysis_Air[item][4] = Wave_Region
            Spectral_Lines_For_Analysis_Air[item][5] = Flux_Region
            Spectral_Lines_For_Analysis_Air[item][6] = Continuum[0]
            Spectral_Lines_For_Analysis_Air[item][7] = Continuum[1]

            if np.isnan(pEQW_LinCon_Spec):
                Spectral_Lines_For_Analysis_Air[item][8] = -9999
            else:
                Spectral_Lines_For_Analysis_Air[item][8] = pEQW_LinCon_Spec

            if np.isnan(Line_Flux):
                Spectral_Lines_For_Analysis_Air[item][16] = -9999
            else:
                Spectral_Lines_For_Analysis_Air[item][16] = Line_Flux

            ##############################
            # Candidate Conditions Checks
            ##############################

            if Spectral_Lines_For_Analysis_Air_Names[ii] in Spectral_Lines_For_Scoring_Air:

                LineScore = Hirogen_Functions.lin_con_candidate_line_identifier_verbose(
                    line_name=Spectral_Lines_For_Analysis_Air_Names[ii],
                    line_peqw=Spectral_Lines_For_Analysis_Air[item][8],
                    peqw_threshold=LineDetection_pEQW_Threshold,
                    continuum_removed_flux=Continuum[1],
                    shift_points=Shift_Region,
                    peak_threshold=LineDetection_Max_Threshold,
                    above_continuum_threshold=LineDetection_Max_Above_Av_Continua_Threshold,
                    peak_max_region=LineDetection_Peak_Tolerance,
                    peak_region=Line_Peak_Location_Region,
                    feature_region=3000,
                    peak_region_minima_threshold=Line_Peak_Region_Minima_Threshold
                )
                print("\n")

                # Score, Region Mean, Peak Max, Pass/Fail
                Candidate_Score = Candidate_Score + LineScore[0]
                Spectral_Lines_For_Analysis_Air[item][9] = LineScore[4]
                Spectral_Lines_For_Analysis_Air[item][10] = LineScore[2]
                Spectral_Lines_For_Analysis_Air[item][11] = LineScore[7]

                if Line_Report:
                    line_scoring_report.write(f'{Spectral_Lines_For_Analysis_Air_Names[ii]}'
                                              f'\t{Spectral_Lines_For_Analysis_Air[item][8]}\t{LineScore[1]}\t'
                                              f'{LineScore[2]}\t{LineScore[3]}\t{LineScore[4]}\t{LineScore[5]}\t'
                                              f'{LineScore[6]}\t{LineScore[8]}\n')

            else:
                Spectral_Lines_For_Analysis_Air[item][9] = np.nan
                Spectral_Lines_For_Analysis_Air[item][10] = np.nan
                Spectral_Lines_For_Analysis_Air[item][11] = np.nan

    ######
    # Score Check
    # Does a quick check to make sure that object score here matches the one found by the analysis script:
    # It SHOULD always match if the criteria were all the same
    ######

    if not ignore_score_check:

        if Candidate_Score != Recorded_Database_Score:
            print("Candidate Score does NOT match the value recorded in the database.\n"
                  f"ID: {ObjectName}\n"
                  f"Current Score: {Candidate_Score}\tRecorded Score: {Recorded_Database_Score}"
                  "\nThis is likely due to a parameter redefinition between the running of this script and the "
                  "last time the analysis script was run."
                  "\nIt can also be caused by a change or enabled/disabled spectral smoothing"
                  "\nI 'suggest' the analysis script be rerun now.")
            # sys.exit()

    ################################
    # Main 'Snapshot' Plot Config - Generated once per spectrum
    ################################

    snap_shot_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        object_spectrum=[Rest_Wavelength, Flux],
        database_data=Candidate_Data[Spectrum_Number],
        lines_to_plot=Spectral_Lines_For_Snapshot_Plot_1,
        line_labels=Line_Labels_1,
        line_data=Spectral_Lines_For_Analysis_Air,
        lines_for_scoring=Spectral_Lines_For_Scoring_Air,
        lick_delta_index=Lick_Hdelta[Spectrum_Number],
        lick_delta_index_err=Lick_Hdelta_Err[Spectrum_Number],
        score=Candidate_Score,
        save_path=PlotsSavePath,
        text_flag=True
    )

    ################################
    # Spectrum only version of the main plot
    ################################

    snap_shot_figure_spec_only(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        object_spectrum=[Rest_Wavelength, Flux],
        database_data=Candidate_Data[Spectrum_Number],
        lines_to_plot=Spectral_Lines_For_Snapshot_Plot_1,
        line_labels=Line_Labels_1,
        line_data=Spectral_Lines_For_Analysis_Air,
        lines_for_scoring=Spectral_Lines_For_Scoring_Air,
        lick_delta_index=Lick_Hdelta[Spectrum_Number],
        lick_delta_index_err=Lick_Hdelta_Err[Spectrum_Number],
        score=Candidate_Score,
        save_path=PlotsSavePath,
        text_flag=False
    )

    ################################
    # Presentation version of the main plot
    ################################

    presentation_snap_shot_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        object_spectrum=[Rest_Wavelength, Flux],
        database_data=Candidate_Data[Spectrum_Number],
        lines_to_plot=Comparison_Lines,
        line_labels=Comparison_Line_Labels,
        line_data=Spectral_Lines_For_Analysis_Air,
        lines_for_scoring=Spectral_Lines_For_Scoring_Air,
        lick_delta_index=Lick_Hdelta[Spectrum_Number],
        lick_delta_index_err=Lick_Hdelta_Err[Spectrum_Number],
        score=Candidate_Score,
        save_path=PlotsSavePath,
        zoom_lines=Zoom_in_Presentation_Lines,
        zoom_line_labels=Zoom_in_Presentation_Line_Labels
    )

    ################################
    # Secondary 'Snapshot' Plot Config - Generated once per spectrum
    ################################

    secondary_snap_shot_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        object_spectrum=[Rest_Wavelength, Flux],
        database_data=Candidate_Data[Spectrum_Number],
        lines_to_plot=Spectral_Lines_For_Snapshot_Plot_2,
        line_labels=Line_Labels_2,
        line_data=Spectral_Lines_For_Analysis_Air,
        score=Candidate_Score,
        save_path=PlotsSavePath,
    )

    ################################
    # Tertiary 'Snapshot' Plot Config - Generated once per spectrum
    ################################

    tertiary_snap_shot_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        object_spectrum=[Rest_Wavelength, Flux],
        database_data=Candidate_Data[Spectrum_Number],
        lines_to_plot=Spectral_Lines_For_Snapshot_Plot_3,
        line_labels=Line_Labels_3,
        line_data=Spectral_Lines_For_Analysis_Air,
        score=Candidate_Score,
        save_path=PlotsSavePath,
    )

    ###############
    # BPT Diagram
    ###############

    bpt_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        database_data=Candidate_Data[Spectrum_Number],
        save_path=PlotsSavePath,
    )

    ###############
    # Full BPT Diagnostic Set
    ###############

    bpt_diagnostic_set_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        database_data=Candidate_Data[Spectrum_Number],
        save_path=PlotsSavePath,
    )

    ###############
    # Line Diagnostic Plot
    ###############
    # For now only plots a one point scatter plot of the two calculated Lick Indices but can be extended when more
    # ratios or diagnostics are desired

    line_diagnostic_figure(
        object_name=ObjectName,
        object_id=Object_Spec_Id,
        database_data=Candidate_Data[Spectrum_Number],
        save_path=PlotsSavePath,
    )

    if Line_Report:
        line_scoring_report.close()

    plt.close('all')
    gc.collect()

    # Failsafe Cut until the memory leak is sorted
    if Spectrum_Number == 2000:
        endT = time.time()
        executionT = endT - startT
        print(f"Failsafe code exit on Object Number 2000 tripped")
        print(f"Script execution time: {executionT:.2f}s")
        sys.exit()

    # print("\n\n\n")
    # sys.exit()

endT = time.time()
executionT = endT - startT

print(f"Script execution time: {executionT:.2f}s")